# wps_full_rvic

#### wps_full_rvic is a process that runs the [parameters](https://rvic.readthedocs.io/en/latest/user-guide/parameters/) and [convolution](https://rvic.readthedocs.io/en/latest/user-guide/convolution/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import os
from datetime import datetime
from wps_tools.testing import get_target_url
from netCDF4 import Dataset
from rvic.core.config import read_config
from importlib.resources import files

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("osprey")
print(f"Using osprey on {url}")

Using osprey on https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps


In [3]:
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
osprey.full_rvic?

Signature:
osprey.full_rvic(
    case_id,
    grid_id,
    run_startdate,
    stop_date,
    pour_points_csv=None,
    uh_box_csv=None,
    routing=None,
    domain=None,
    input_forcings=None,
    loglevel='INFO',
    version=True,
    np=1,
    params_config_file=None,
    params_config_dict=None,
    convolve_config_file=None,
    convolve_config_dict=None,
)
Docstring:
Run full RVIC process combining Parameters and Convolution modules

Parameters
----------
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
version : boolean
    Return RVIC version string
np : integer
    Number of processors used to run job
case_id : string
    Case ID for the RVIC process
grid_id : string
    Routing domain grid shortname
run_startdate : string
    Run start date (yyyy-mm-dd-hh). Only used for startup and drystart runs.
stop_date : string
    Run stop date based on STOP_OPTION
pour_points_csv : ComplexData:mimetype:`text/plain`, :mimetype:`text/csv`
 

#### We can use the docstring to ensure we provide the appropriate parameters.

In [5]:
# run full_rvic with only required inputs
output_basic = osprey.full_rvic(
    case_id = "sample",
    grid_id = "COLUMBIA",
    run_startdate = "2012-12-01-00",
    stop_date = "2012-12-31",
    pour_points_csv = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/fileServer/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_pour.txt",
    uh_box_csv = (files("tests") / "data/samples/uhbox.csv").read_text(),
    routing = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_flow_parameters.nc",
    domain = str((files("tests") / "data/samples/sample_routing_domain.nc").resolve()),
    input_forcings = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/columbia_vicset2.nc",
)
# Use asobj=True to access the data from the output file
basic_data = output_basic.get(asobj=True)[0]

In [6]:
# run full_rvic with local configuration files
output_config_files = osprey.full_rvic(
    case_id = "sample",
    grid_id = "COLUMBIA",
    run_startdate = "2012-12-01-00",
    stop_date = "2012-12-31",
    pour_points_csv = (files("tests") / "data/samples/sample_pour.txt").read_text(),
    uh_box_csv = (files("tests") / "data/samples/uhbox.csv").read_text(),
    routing = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_flow_parameters.nc",
    domain = str((files("tests") / "data/samples/sample_routing_domain.nc").resolve()),
    input_forcings = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/columbia_vicset2.nc",
    params_config_file = str((files("tests") / "data/configs/parameters.cfg").resolve()),
    convolve_config_file = str((files("tests") / "data/configs/convolve.cfg").resolve()),
)
config_files_data = output_config_files.get(asobj=True)[0]

In [7]:
# run parameters with configuration dictionaries
output_config_dicts = osprey.full_rvic(
    case_id = "sample",
    grid_id = "COLUMBIA",
    run_startdate = "2012-12-01-00",
    stop_date = "2012-12-31",
    pour_points_csv = (files("tests") / "data/samples/sample_pour.txt").read_text(),
    uh_box_csv = (files("tests") / "data/samples/uhbox.csv").read_text(),
    routing = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/sample_flow_parameters.nc",
     domain = str((files("tests") / "data/samples/sample_routing_domain.nc").resolve()),
    input_forcings = "https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/climate_explorer_data_prep/hydro/sample_data/set4/columbia_vicset2.nc",
    params_config_dict = {
        "OPTIONS": {
            "LOG_LEVEL": "CRITICAL",
        },
    },
    convolve_config_dict = {
        "OPTIONS": {
            "CASESTR": "Historical",
        },
    },
)
config_dicts_data = output_config_dicts.get(asobj=True)[0]

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [8]:
params_config = read_config("tests/data/configs/parameters.cfg")
convolve_config = read_config("tests/data/configs/convolve.cfg")

# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC history file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d").zfill(2),
    "RvicFdrFile": "sample_flow_parameters.nc",
}


In [9]:
def test_full_rvic_output(output_data, expected):
    # Metadata of the final output of RVIC parameters + convolution 
    metadata = {
        "title": output_data.title,
        "year": output_data.history.split()[5],
        "month": output_data.history.split()[2].zfill(2),
        "day": output_data.history.split()[3].zfill(2),
        "RvicFdrFile": output_data.RvicFdrFile,
    }

    assert metadata == expected

In [10]:
test_full_rvic_output(basic_data, expected)
test_full_rvic_output(config_files_data, expected)
test_full_rvic_output(config_dicts_data, expected)